## Import necessary library

In [1]:
import os
import glob
import pickle
import zipfile
import numpy as np
import pandas as pd
from tqdm import tqdm
import scipy.signal as scisig
from scipy.signal import welch
from collections import Counter

## Extrack zipfile from S[num] to E4 folder

In [2]:
base_path = "/Users/wan_aubel/Desktop/focus-detection/focus-detection-using-physiological-data/data/raw/WESAD"

In [3]:
for folder in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder)
    if os.path.isdir(folder_path) and folder.startswith('S'):
        zip_file = [f for f in os.listdir(folder_path) if f.endswith(".zip")]
        zip_path = os.path.join(folder_path, zip_file[0])
        extract_path = os.path.join(folder_path, "E4")

        os.makedirs(extract_path)

        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)

## Parse information of each subject

In [4]:
def parse_subject_file(filepath):
    with open(filepath, 'r') as f:
        lines = f.read().splitlines()
    
    data = {}

    for line in lines:
        if not line.strip() or line.startswith("###"):
            continue
        if ":" in line:
            key, value = line.split(":", 1)
            data[key.strip()] = value.strip()
        elif "?" in line:
            key, value = line.rsplit(" ", 1)
            data[key.strip()] = value.strip()
        else:
            data["Notes"] = line.strip()

    return data

In [5]:
base_path = "/Users/wan_aubel/Desktop/focus-detection/focus-detection-using-physiological-data/data/raw/WESAD"
subject_data = []

In [6]:
for folder in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder)
    if os.path.isdir(folder_path) and folder.startswith('S'):
        subject_id = folder
        readme_path = os.path.join(folder_path, f"{subject_id}_readme.txt")

        data = parse_subject_file(readme_path)
        data["subject"] = subject_id

        subject_data.append(data)

In [7]:
subject_data.sort(key=lambda x: int(x["subject"][1:]))

In [8]:
df = pd.DataFrame(subject_data)
df = df[["subject"] + [col for col in df.columns if col != "subject"]]

In [9]:
df.head(5)

,subject,Age,Height (cm),Weight (kg),Gender,Dominant hand,Did you drink coffee today?,Did you drink coffee within the last hour?,Did you do any sports today?,Are you a smoker?,Did you smoke within the last hour?,Do you feel ill today?,Notes,Stress condition / TSST interview part
0,S2,27,175,80,male,right,NO,NO,NO,NO,NO,NO,The RespiBAN temperature sensor was not fully ...,NaN
1,S3,27,173,69,male,right,NO,NO,NO,NO,NO,NO,Subject provided a valence label of 7 after th...,NaN
2,S4,25,175,90,male,right,NO,NO,NO,NO,NO,NO,-,NaN
3,S5,35,189,80,male,right,YES,NO,NO,NO,NO,NO,Subject might have fallen asleep during the fi...,NaN
4,S6,27,170,66,male,right,YES,NO,NO,YES,NO,NO,Subject claimed that he had a stressfull week ...,subject was not really stressed as he encounte...


In [10]:
df = df.drop(["Notes", "Stress condition / TSST interview part"], axis=1)
df.head(5)

,subject,Age,Height (cm),Weight (kg),Gender,Dominant hand,Did you drink coffee today?,Did you drink coffee within the last hour?,Did you do any sports today?,Are you a smoker?,Did you smoke within the last hour?,Do you feel ill today?
0,S2,27,175,80,male,right,NO,NO,NO,NO,NO,NO
1,S3,27,173,69,male,right,NO,NO,NO,NO,NO,NO
2,S4,25,175,90,male,right,NO,NO,NO,NO,NO,NO
3,S5,35,189,80,male,right,YES,NO,NO,NO,NO,NO
4,S6,27,170,66,male,right,YES,NO,NO,YES,NO,NO


In [11]:
save_path = "/Users/wan_aubel/Desktop/focus-detection/focus-detection-using-physiological-data/data/processed/subject_info.csv"
df.to_csv(save_path, index=False)

## Extract and gather physiological data from each subject

In [12]:
class SubjectData:
    def __init__(self, main_path, subject_number):
        self.name = f'S{subject_number}'
        with open(os.path.join(main_path, self.name, self.name + '.pkl'), 'rb') as file:
            self.data = pickle.load(file, encoding='latin1')
        self.bvp = self.data['signal']['wrist']['BVP']
        self.acc = self.data['signal']['wrist']['ACC']
        self.eda = self.data['signal']['wrist']['EDA']
        self.resp = self.data['signal']['chest']['Resp']
        self.labels = self.data['label']

### Define feature calculate function

In [13]:
def bvp_to_hrv(bvp_signal, fs):
    # Detect peaks
    peaks, _ = scisig.find_peaks(bvp_signal, distance=int(fs * 0.4))

    if len(peaks) < 3:
        return pd.DataFrame()
    
    # คำนวณ IBI
    ibi = np.diff(peaks) / fs * 1000 # ms
    rr_diff = np.diff(ibi)
    
    # Time axis for interpolation
    ibi_time = np.cumsum(ibi) / 1000 # sec
    interp_time = np.arange(0, ibi_time[-1], 1/0.4)
    ibi_interp = np.interp(interp_time, ibi_time, ibi)

    # คำนวณ HR
    hr = (60 * 1000) / ibi # bpm

    # HRV  metrics
    rmssd = np.sqrt(np.mean(rr_diff ** 2)) if len(rr_diff) > 0 else np.nan
    sdnn = np.std(ibi) if len(ibi) > 1 else np.nan
    pNN50 = np.sum(np.abs(rr_diff) > 50) / len(rr_diff) * 100 if len(rr_diff) > 0 else np.nan

    # Frequency domain
    f, pxx = welch(ibi_interp, fs=0.4)
    lf = np.trapz(pxx[(f >= 0.04) & (f <= 0.15)], f[(f >= 0.04) & (f <= 0.15)])
    hf = np.trapz(pxx[(f > 0.15) & (f <= 0.4)], f[(f > 0.15) & (f <= 0.4)])
    lf_hf_ratio = lf / hf if hf != 0 else np.nan

    # Alighn HR/IBI timestamsp (start at 2nd Beats)
    timestamps = peaks[1:] / fs

    return pd.DataFrame({
        'timestamps': pd.to_datetime(timestamps, unit='s'),
        'HR': hr,
        'IBI': ibi,
        'RMSSD': rmssd,
        'SDNN': sdnn,
        'pNN50': pNN50,
        'lf/hf': lf_hf_ratio
    })

In [14]:
def extract_resp_features(resp_signal, fs):
    peaks, _ = scisig.find_peaks(resp_signal, distance=fs * 2)
    if len(peaks) < 2:
        return {'RESP_rate': np.nan, 'RESP_regularity': np.nan}
    
    ibi = np.diff(peaks) / fs
    resp_rate = 60 / np.mean(ibi) if np.mean(ibi) > 0 else np.nan
    regularity = 1 / np.std(ibi) if np.std(ibi) > 0 else np.nan
    return {
        'RESP_rate': resp_rate,
        'RESP_regularity': regularity
        }

def extract_eda_features(eda_signal):
    x = np.arange(len(eda_signal))
    slope = float(np.polyfit(x, eda_signal, 1)[0]) if len(eda_signal) > 1 else np.nan
    return {
        'EDA_mean': np.mean(eda_signal),
        'EDA_std': np.std(eda_signal),
        'EDA_slope': slope
        }

### Extract feature and calculate

In [15]:
def feature_extract(subject_id, WINDOW_IN_SECONDS):
    subject = SubjectData(main_path="../../data/raw/WESAD", subject_number=subject_id)

    # Signals
    bvp = subject.bvp.flatten()
    acc = subject.acc
    eda = subject.eda
    resp = subject.resp
    labels = subject.labels

    # Windows
    window_len = fs_dict['label'] * WINDOW_IN_SECONDS
    total_len = len(labels)
    n_windows = total_len // window_len

    all_window = []

    for i in range(n_windows):
        start = i * window_len
        end = (i + 1) * window_len
        
        '''# label majority vote (optional, not used in clustering)
        label_window = labels[start:end]
        label = Counter(label_window).most_common(1)[0][0]'''

        # majority vote label
        label_window = labels[start:end]
        label_window = [l for l in label_window if l in [1, 2, 3, 4]]
        if len(label_window) == 0:
            continue
        label = Counter(label_window).most_common(1)[0][0]

        # --- ACC ---
        acc_window = acc[start * fs_dict['ACC'] // fs_dict['label']: end * fs_dict['ACC'] // fs_dict['label'], :]
        if acc_window.shape[0] == 0: continue
        acc_x, acc_y, acc_z = acc_window[:, 0], acc_window[:, 1], acc_window[:, 2]
        net_acc = np.sqrt(acc_x ** 2 + acc_y ** 2 + acc_z ** 2)
        acc_features = {
            'ACC_x_mean': np.mean(acc_x),
            'ACC_y_mean': np.mean(acc_y),
            'ACC_z_mean': np.mean(acc_z),
            'net_acc_mean': np.mean(net_acc),
            'net_acc_std': np.std(net_acc)
        }

        # --- BVP / HRV ---
        bvp_window = bvp[start * fs_dict['BVP'] // fs_dict['label']: end * fs_dict['BVP'] // fs_dict['label']]
        hrv_df = bvp_to_hrv(bvp_window, fs_dict['BVP'])
        if hrv_df.empty: continue
        hrv_mean = hrv_df[['HR', 'IBI', 'RMSSD', 'SDNN', 'pNN50', 'lf/hf']].mean()

        # --- EDA ---
        eda_window = eda[start * fs_dict['EDA'] // fs_dict['label']: end * fs_dict['EDA'] // fs_dict['label']]
        eda_features = extract_eda_features(eda_window)

        # --- RESP ---
        resp_window = resp[start * fs_dict['Resp'] // fs_dict['label']: end * fs_dict['Resp'] // fs_dict['label']]
        if resp_window.ndim > 1:
            resp_window = resp_window.flatten()
        resp_features = extract_resp_features(resp_window, fs_dict['Resp'])

        data = {
            **acc_features,
            **eda_features,
            **resp_features,
            'HR': hrv_mean['HR'],
            'IBI': hrv_mean['IBI'],
            'RMSSD': hrv_mean['RMSSD'],
            'SDNN': hrv_mean['SDNN'],
            'pNN50': hrv_mean['pNN50'],
            'lf/hf': hrv_mean['lf/hf'],
            'label': label,
            'subject': subject_id
        }
        all_window.append(data)

    df = pd.DataFrame(all_window)
    df.to_csv(f'{save_path}/S{subject_id}.csv', index=False)
    print(f'Subject {subject_id} processed with {len(df)} samples.')

### Define sampling rate and save path

In [16]:
# Sampling rates
fs_dict = {'ACC': 50, 'BVP': 64, 'EDA': 4, 'Resp': 700, 'label': 700}
subject_id = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]
WINDOW_IN_SECONDS = [30, 60, 300, 600]


# Save path
main_path = "../../data/processed/WESAD/feature_extracted/"

In [17]:
for wind_size in WINDOW_IN_SECONDS:
    for id in subject_id:
        save_path = main_path + f'{wind_size}s/'
        feature_extract(id, WINDOW_IN_SECONDS=wind_size)

/opt/homebrew/lib/python3.11/site-packages/scipy/signal/_spectral_py.py:790: UserWarning: nperseg = 256 is greater than input length  = 12, using nperseg = 12
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
/var/folders/68/zr220k7d2xj94vd30c_16wtc0000gn/T/ipykernel_55989/2629634632.py:27: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  lf = np.trapz(pxx[(f >= 0.04) & (f <= 0.15)], f[(f >= 0.04) & (f <= 0.15)])
/var/folders/68/zr220k7d2xj94vd30c_16wtc0000gn/T/ipykernel_55989/2629634632.py:28: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  hf = np.trapz(pxx[(f > 0.15) & (f <= 0.4)], f[(f > 0.15) & (f <= 0.4)])
/var/folders/68/zr220k7d2xj94vd30c_16wtc0000gn/T/ipykernel_55989/4132853386.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will err

Subject 2 processed with 61 samples.


/opt/homebrew/lib/python3.11/site-packages/scipy/signal/_spectral_py.py:790: UserWarning: nperseg = 256 is greater than input length  = 12, using nperseg = 12
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
/var/folders/68/zr220k7d2xj94vd30c_16wtc0000gn/T/ipykernel_55989/2629634632.py:27: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  lf = np.trapz(pxx[(f >= 0.04) & (f <= 0.15)], f[(f >= 0.04) & (f <= 0.15)])
/var/folders/68/zr220k7d2xj94vd30c_16wtc0000gn/T/ipykernel_55989/2629634632.py:28: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  hf = np.trapz(pxx[(f > 0.15) & (f <= 0.4)], f[(f > 0.15) & (f <= 0.4)])
/var/folders/68/zr220k7d2xj94vd30c_16wtc0000gn/T/ipykernel_55989/4132853386.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will err

Subject 3 processed with 62 samples.


/opt/homebrew/lib/python3.11/site-packages/scipy/signal/_spectral_py.py:790: UserWarning: nperseg = 256 is greater than input length  = 12, using nperseg = 12
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
/var/folders/68/zr220k7d2xj94vd30c_16wtc0000gn/T/ipykernel_55989/2629634632.py:27: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  lf = np.trapz(pxx[(f >= 0.04) & (f <= 0.15)], f[(f >= 0.04) & (f <= 0.15)])
/var/folders/68/zr220k7d2xj94vd30c_16wtc0000gn/T/ipykernel_55989/2629634632.py:28: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  hf = np.trapz(pxx[(f > 0.15) & (f <= 0.4)], f[(f > 0.15) & (f <= 0.4)])
/var/folders/68/zr220k7d2xj94vd30c_16wtc0000gn/T/ipykernel_55989/4132853386.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will err

...

Subject 15 processed with 6 samples.


/opt/homebrew/lib/python3.11/site-packages/scipy/signal/_spectral_py.py:790: UserWarning: nperseg = 256 is greater than input length  = 240, using nperseg = 240
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
/var/folders/68/zr220k7d2xj94vd30c_16wtc0000gn/T/ipykernel_55989/2629634632.py:27: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  lf = np.trapz(pxx[(f >= 0.04) & (f <= 0.15)], f[(f >= 0.04) & (f <= 0.15)])
/var/folders/68/zr220k7d2xj94vd30c_16wtc0000gn/T/ipykernel_55989/2629634632.py:28: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  hf = np.trapz(pxx[(f > 0.15) & (f <= 0.4)], f[(f > 0.15) & (f <= 0.4)])
/var/folders/68/zr220k7d2xj94vd30c_16wtc0000gn/T/ipykernel_55989/4132853386.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will e

Subject 16 processed with 6 samples.


/opt/homebrew/lib/python3.11/site-packages/scipy/signal/_spectral_py.py:790: UserWarning: nperseg = 256 is greater than input length  = 240, using nperseg = 240
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
/var/folders/68/zr220k7d2xj94vd30c_16wtc0000gn/T/ipykernel_55989/2629634632.py:27: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  lf = np.trapz(pxx[(f >= 0.04) & (f <= 0.15)], f[(f >= 0.04) & (f <= 0.15)])
/var/folders/68/zr220k7d2xj94vd30c_16wtc0000gn/T/ipykernel_55989/2629634632.py:28: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  hf = np.trapz(pxx[(f > 0.15) & (f <= 0.4)], f[(f > 0.15) & (f <= 0.4)])
/var/folders/68/zr220k7d2xj94vd30c_16wtc0000gn/T/ipykernel_55989/4132853386.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will e

Subject 17 processed with 7 samples.


## Refined new focus-label

In [18]:
def quest_load(subject_id):
    subject_path = f'../../data/raw/WESAD/S{subject_id}/S{subject_id}_quest.csv'

    with open(subject_path, 'r') as f:
        lines = f.readlines()

    order_line = [l for l in lines if l.startswith('# ORDER')][0]
    order = order_line.strip().split(';')[1:-1]
    order = [o.strip().lower() for o in order if o.strip() not in ['bread','fread','sread']]

    stai_dict = {}
    valence_dict = {}
    arousal_dict = {}

    stai_lines = [l for l in lines if l.startswith('# STAI')]
    dim_lines = [l for l in lines if l.startswith('# DIM')]

    stai_score = np.array([[int(i) for i in l.strip().split(';')[1:-1] if i.strip().isdigit()] for l in stai_lines])
    dim_score = np.array([[int(i) for i in l.strip().split(';')[1:-1] if i.strip().isdigit()] for l in dim_lines])

    # DIM score
    valence_score = dim_score[:, 0]
    arousal_score = dim_score[:, 1]

    min_len = min(len(order), len(valence_score))

    valence_score = valence_score[:min_len]
    arousal_score = arousal_score[:min_len]

    # STAI score
    for i, task in enumerate(order[:min_len]):
        if task.startswith('medi'):
            task_key = 'meditation'
        else:
            task_key = task

        positives = (stai_score[:, 0] + stai_score[:, 3] + stai_score[:, 5]) / 3
        negatives = (stai_score[:, 1] + stai_score[:, 2] + stai_score[:, 4]) / 3

        scored_stai = negatives[i] - positives[i]

        valence_dict[task_key] = valence_score[i]
        arousal_dict[task_key] = arousal_score[i]
        stai_dict[task_key] = scored_stai

    return{
        'valence': valence_dict,
        'arousal': arousal_dict,
        'stai': stai_dict
    }

In [19]:
def refined_label(lbl, dim_arousal, dim_valence, stai_score):
    if lbl in [1, 4]:
        if (dim_arousal <= 3) and (dim_valence >= 5):
            return 1
        else:
            return 0
        
    elif lbl == 2:
        if stai_score <= 0:
            if dim_arousal >= 5:
                return 1
            else:
                return 0
        else:
            return 0
        
    else:
        return 0

In [20]:
label_to_task = {
    1: 'base',
    2: 'tsst',
    3: 'fun',
    4: 'meditation'
}

In [21]:
for wind_size in [30, 60, 300, 600]:
    all_file = glob.glob(f'../../data/processed/WESAD/feature_extracted/{wind_size}s/S*.csv')
    df_all = pd.concat([pd.read_csv(file) for file in all_file], ignore_index=True, axis=0)
    df_all['focus_label'] = -1

    quest_dict = {}

    for s in df_all['subject'].unique():
        quest_dict[s] = quest_load(s)

    for i, row in tqdm(df_all.iterrows(), total=len(df_all)):
        subj = int(row['subject'])
        lbl = int(row['label'])

        if lbl not in label_to_task:
            continue

        task_name = label_to_task[lbl]
        quest = quest_dict[subj]

        # ดึงคะแนนที่ต้องใช้
        dim_arousal = quest['arousal'][task_name]
        dim_valence = quest['valence'][task_name]
        stai_score = quest['stai'][task_name] if lbl == 2 else None

        # คำนวณ label
        focus = refined_label(lbl, dim_arousal, dim_valence, stai_score)

        # บันทึกลง dataframe
        df_all.at[i, 'focus_label'] = focus

    df_all.to_csv(f'../../data/processed/WESAD/data_processed/{wind_size}s/all_data.csv')

100%|██████████| 91/91 [00:00<00:00, 62325.55it/s]
